In [12]:
!ls

BERTSeqClassification  pvpod.yaml		   torchserve-pv.yaml
config		       pv.yaml			   torchserve-s3.yml
model-conv.ipynb       s3-secret.yaml		   ts_test.ipynb
model-store	       torchserve-basic.yaml	   ts.yaml
pvc.yaml	       torchserve-predictor.ipynb


In [13]:
!kubectl apply -f pv.yaml -n kfserving-test

persistentvolume/model-pv-volume unchanged


In [16]:
!kubectl apply -f pvc.yaml -n kfserving-test

persistentvolumeclaim/model-pv-claim created


In [90]:
!kubectl apply -f pvpod.yaml -n kfserving-test

pod/model-store-pod created


In [54]:
!kubectl exec -t model-store-pod -c model-store -n kfserving-test -- mkdir /pv/model-store/
!kubectl exec -t model-store-pod -c model-store -n kfserving-test -- mkdir /pv/config/

mkdir: cannot create directory '/pv/model-store/': File exists
command terminated with exit code 1
mkdir: cannot create directory '/pv/config/': File exists
command terminated with exit code 1


In [91]:
!kubectl cp model-store model-store-pod:/pv/ -c model-store -n kfserving-test

In [56]:
!kubectl cp model-store/properties.json model-store-pod:/pv/model-store/properties.json -c model-store -n kfserving-test

In [151]:
#!kubectl exec -it model-store-pod -n kfserving-test -- cat /pv/model-store/mnist/mnist_handler.py

In [61]:
!kubectl exec -it model-store-pod -n kfserving-test -- ls -lR /pv/model-store/

Error from server (NotFound): pods "model-store-pod" not found


In [74]:
!kubectl exec -it model-store-pod -n kfserving-test -- cat /pv/config/config.properties

inference_address=http://0.0.0.0:8080
management_address=http://0.0.0.0:8081
number_of_netty_threads=32
job_queue_size=1000
model_store="/home/model-server/shared/model-store"
model_snapshot={"name":"startup.cfg","modelCount":"1","models":{"mnist":{"1.0":{"defaultVersion":true,"marName":"mnist.mar","minWorkers":"1","maxWorkers":"3","batchSize":"1","maxBatchDelay":"100","responseTimeout":"120"}}}}


In [92]:
!kubectl delete -f pvpod.yaml -n kfserving-test

pod "model-store-pod" deleted


In [58]:
!kubectl apply -f ts.yaml -n kfserving-test

pod/model-conv-pod created


In [62]:
!kubectl exec -it model-conv-pod -n kfserving-test -- ls -lR /home/model-server/shared/model-store/

/home/model-server/shared/model-store/:
total 4372
drwxrwxrwx 2 root         root            4096 Sep 14 06:20 mnist
-rw-r--r-- 1 model-server model-server 4465323 Sep 14 06:22 mnist.mar
-rwxrwxrwx 1 model-server model-server     287 Sep 14 06:20 properties.json

/home/model-server/shared/model-store/mnist:
total 4700
-rwxrwxrwx 1 model-server model-server     757 Sep 14 06:20 mnist.py
-rwxrwxrwx 1 model-server model-server 4800893 Sep 14 06:20 mnist_cnn.pt
-rwxrwxrwx 1 model-server model-server     693 Sep 14 06:20 mnist_handler.py


In [64]:
!kubectl exec -it model-conv-pod -n kfserving-test -- cat /home/model-server/shared/config/config.properties

inference_address=http://0.0.0.0:8080
management_address=http://0.0.0.0:8081
number_of_netty_threads=32
job_queue_size=1000
model_store="/home/model-server/shared/model-store"
model_snapshot={"name":"startup.cfg","modelCount":"1","models":{"mnist":{"1.0":{"defaultVersion":true,"marName":"mnist.mar","minWorkers":"1","maxWorkers":"3","batchSize":"1","maxBatchDelay":"100","responseTimeout":"120"}}}}


In [65]:
!kubectl delete -f ts.yaml -n kfserving-test

pod "model-conv-pod" deleted


In [79]:
!cat torchserve-pv.yaml

apiVersion: serving.kubeflow.org/v1alpha2
kind: InferenceService
metadata:
  labels:
    controller-tools.k8s.io: "1.0"
  name: ts-sample
spec:
  default:
    predictor:
      torchserve:
        storageUri: "pvc://model-pv-claim"
        # resources:
        #   limits:
        #     nvidia.com/gpu: 1

In [97]:
!kubectl apply -f torchserve-pv.yaml -n kfserving-test 

inferenceservice.serving.kubeflow.org/ts-sample created


In [96]:
!kubectl delete -f torchserve-pv.yaml -n kfserving-test

inferenceservice.serving.kubeflow.org "ts-sample" deleted
